## RNN

### Theory

$$
\begin{aligned}
\bar{{\bf h}}^{(t)}=&{\bf W}_{{\bf x}{\bf h}}{\bf x}^{(t)}+{\bf W}_{{\bf h}{\bf h}}{\bf h}^{(t-1)}+ {\bf b}_{\bf h}\\
{\bf h}^{(t)} =& \tanh\left(\bar{{\bf h}}^{(t)}\right)\\
\bar{{\bf o}}^{(t)}=&{\bf W}_{{\bf h}{\bf o}}{\bf h}^{(t)}+{\bf b}_{\bf o}\\
{\bf o}^{(t)} =& f\left(\bar{{\bf o}}^{(t)}\right)\\
\mathcal{L}=&\sum_{t=1}^T\sum_{i=1}^d -y^{(t)}_i \log\left(o^{(t)}_i\right)\\
=&\sum_{t=1}^T-\log\left(o^{(t)}_{i_*^{(t)}}\right)\\
\text{where }&{i_*^{(t)}}=\arg\max_i\{y_i^{(t)}\}\text{ with }y_{i_*^{(t)}}^{(t)}=1 \\
&{\bf x}^{(t)}\in \mathbb{R}^d,{\bf h}^{(t)}\in \mathbb{R}^m, {\bf o}^{(t)}\in \mathbb{R}^d\\
&{\bf W}_{{\bf x}{\bf h}}\in\mathbb{R}^{m\times d},{\bf W}_{{\bf h}{\bf h}}\in\mathbb{R}^{m\times m},{\bf W}_{{\bf h}{\bf o}}\in \mathbb{R}^{d\times m},{\bf b}_{\bf h}\in \mathbb{R}^m,{\bf b}_{\bf o}\in \mathbb{R}^d\\
\frac{\partial \mathcal{L}}{\partial \bar{o}^{(t)}_j}&=\frac{\partial \mathcal{L}}{\partial {o}^{(t)}_{i_*^{(t)}}}\frac{\partial {o}^{(t)}_{i_*^{(t)}}}{\partial \bar{o}^{(t)}_j}=-\frac{1}{o_{i_{*}^{(t)}}^{(t)}}\frac{\mathbb{1}_\left\{j=i^{(t)}_*\right\}e^{\bar{o}^{(t)}_{i_*^{(t)}}}\left(\sum \cdot\right)-e^{\bar{o}^{(t)}_{i_*^{(t)}}}e^{\bar{o}^{(t)}_{j}}}{\left(\sum \cdot\right)^{2}}=-\frac{1}{o_{i_{*}^{(t)}}^{(t)}}\left(\mathbb{1}_\left\{j=i^{(t)}_*\right\}o_{i_{*}^{(t)}}^{(t)}-o_{i_{*}^{(t)}}^{(t)}o_{j}^{(t)}\right)=o_{j}^{(t)}-\mathbb{1}_\left\{j=i^{(t)}_*\right\}\\
&\frac{\partial \mathcal{L}}{\partial {\bf b}_{\bf o}}=\sum_{t=1}^{T} \frac{\partial \mathcal{L}}{\partial \bar{\bf o}^{(t)}}\frac{\partial \bar{\bf o}^{(t)}}{\partial {\bf b}_{\bf o}}=\sum_{t=1}^{T}I^\top\left({\bf o}^{(t)}-{\bf e}_{i^{(t)}_*}\right)=\sum_{t=1}^{T}{\bf o}^{(t)}-{\bf e}_{i^{(t)}_*}\\
&\frac{\partial \mathcal{L}}{\partial {\bf W}_{{\bf h}{\bf o}}}=\sum_{t=1}^{T} \frac{\partial \mathcal{L}}{\partial \bar{\bf o}^{(t)}}\frac{\partial \bar{\bf o}^{(t)}}{\partial {\bf W}_{{\bf h}{\bf o}}}=\sum_{t=1}^{T}\left({\bf o}^{(t)}-{\bf e}_{i^{(t)}_*}\right){{\bf h}^{(t)}}^\top\\
&\frac{\partial \mathcal{L}}{\partial {\bf h}^{(t)}}=\frac{\partial \mathcal{L}}{\partial {\bf h}^{(t+1)}}\frac{\partial {\bf h}^{(t+1)}}{\partial {\bf h}^{(t)}}+\frac{\partial \mathcal{L}}{\partial \bar{\bf o}^{(t)}}\frac{\partial \bar{\bf o}^{(t)}}{\partial {\bf h}^{(t)}}={\bf W}_{{\bf h}{\bf h}}^\top\text{diag}\left({\bf 1}-\left({{\bf h}^{(t+1)}}\right)^2\right)\frac{\partial \mathcal{L}}{\partial {\bf h}^{(t+1)}}+{\bf W}_{{\bf h}{\bf o}}^\top\left({\bf o}^{(t)}-{\bf e}_{i^{(t)}_*}\right)\\
&\frac{\partial \mathcal{L}}{\partial {\bf W}_{{\bf h}{\bf h}}}=\sum_{t=1}^{T} \frac{\partial \mathcal{L}}{\partial {\bf h}^{(t)}}\frac{\partial {\bf h}^{(t)}}{\partial {\bf W}_{{\bf h}{\bf h}}}=\sum_{t=1}^{T} \text{diag}\left({\bf 1}-\left({\bf h}^{(t)}\right)^2\right)\frac{\partial \mathcal{L}}{\partial {\bf h}^{(t)}}\left({\bf h}^{(t-1)}\right)^\top\\
&\frac{\partial \mathcal{L}}{\partial {\bf W}_{{\bf x}{\bf h}}}=\sum_{t=1}^{T} \frac{\partial \mathcal{L}}{\partial {\bf h}^{(t)}}\frac{\partial {\bf h}^{(t)}}{\partial {\bf W}_{{\bf x}{\bf h}}}=\sum_{t=1}^{T} \text{diag}\left({\bf 1}-\left({\bf h}^{(t)}\right)^2\right)\frac{\partial \mathcal{L}}{\partial {\bf h}^{(t)}}\left({\bf x}^{(t)}\right)^\top\\
\end{aligned}
$$

### Scratch

In [11]:
import numpy as np
from tqdm import tqdm
import re

In [168]:
class Reader:
    def __init__(self, fpath):
        self._fpath = fpath
        with open(self._fpath, 'r') as f:
            self.data = f.read()
            self.data = self.data.replace("\n", ' ')
            self.data = self.data.lower()
        self.data = re.sub(r'([\"\'])?([A-z]+[-\']?[A-z]+)?([\"\':,.?!])?', r'\1 \2 \3', self.data).split()
        chars = list(set(self.data))
        self.char_to_ix = {ch:i for i,ch in enumerate(chars)}
        self.ix_to_char = {i:ch for i,ch in enumerate(chars)}
        self.vocab_size = len(chars)

In [176]:
class RNN:
    def __init__(self, vocab_size, learning_rate=0.1, sequence_length=25, hidden_size=100):
        self._learning_rate = learning_rate # learning rate
        self._sequence_length = sequence_length
        self._hidden_size = hidden_size
        self._vocab_size = vocab_size
        np.random.seed(42)
        self._Wxh = np.random.randn(self._hidden_size,self._vocab_size)*0.01
        self._Whh = np.random.randn(self._hidden_size,self._hidden_size)*0.01
        self._Who = np.random.randn(self._vocab_size,self._hidden_size)*0.01
        self._bh = np.zeros(hidden_size)
        self._bo = np.zeros(vocab_size)
    
    def _softmax(self, x):
        ex = np.exp(x-max(x))
        return ex/np.sum(ex)
        
    def loss(self, os, y):
        return sum(-np.log(max(os[t][y[t]],0.1/self._vocab_size)) for t in range(self._sequence_length))
        # mind the \
        # return loss using Xentropy
        
    
    def forward(self, inputs, hprev):
        xs, hs, os = {}, {}, {}
        hs[-1] = hprev
        for t in range(self._sequence_length):
            xs[t] = np.zeros(self._vocab_size)
            xs[t][inputs[t]] = 1
            hs[t] = np.tanh(self._Wxh@xs[t]+self._Whh@hs[t-1]+self._bh)
            os[t] = self._softmax(self._Who@hs[t]+self._bo)
        return xs, hs, os

    
    def backward(self, xs, hs, os, y):
        dbo, dbh = np.zeros_like(self._bo), np.zeros_like(self._bh)
        dWho, dWhh, dWxh = np.zeros_like(self._Who), np.zeros_like(self._Whh), np.zeros_like(self._Wxh)
        dhnext = np.zeros(self._hidden_size)
        for t in range(self._sequence_length-1,-1,-1):
            do = np.zeros(self._vocab_size)+os[t]
            do[y[t]] -= 1
            dh = (1-hs[t]**2)*(self._Who.T@do+dhnext)
            dhnext = self._Whh.T@dh
            dbo += do
            dbh += dh
            dWho += np.outer(do,hs[t])
            dWhh += np.outer(dh,hs[t-1])
            dWxh += np.outer(dh,xs[t])
        for dparam in (dbo, dbh, dWho, dWhh, dWxh):
            np.clip(dparam, -5, 5, out=dparam)
        return dbo, dbh, dWho, dWhh, dWxh
    
    def fit(self, reader, epoch):
        smooth_loss = -np.log(1.0/self._vocab_size)*self._sequence_length
        mbo, mbh = np.zeros_like(self._bo), np.zeros_like(self._bh)
        mWho, mWhh, mWxh = np.zeros_like(self._Who), np.zeros_like(self._Whh), np.zeros_like(self._Wxh)
        
        for _ in tqdm(range(epoch)): 
            n = 0
            hprev = np.zeros(self._hidden_size)
            for p in range(0, len(reader.data)-self._sequence_length, self._sequence_length):
                inputs = [reader.char_to_ix[ch] for ch in reader.data[p:p+self._sequence_length]]
                #print(inputs)
                y = inputs[1:]+[reader.char_to_ix[reader.data[p+self._sequence_length]]]
                #print(y)
                xs, hs, os = self.forward(inputs, hprev)
                dbo, dbh, dWho, dWhh, dWxh = self.backward(xs, hs, os, y)
                #print(dWho)
                for param, dparam, mem in zip((self._bo, self._bh, self._Who, self._Whh, self._Wxh),
                                              (dbo, dbh, dWho, dWhh, dWxh),
                                              (mbo, mbh, mWho, mWhh, mWxh)):
                    mem += dparam*dparam
                    param += -self._learning_rate*dparam/np.sqrt(mem+1e-8)
                hprev = hs[self._sequence_length-1]
                smooth_loss = smooth_loss * 0.999 + self.loss(os, y) * 0.001
                n+=1
                if n%400 == 0:
                    print(f"loss: {smooth_loss}")
                    print(self.predict(reader,'I', self._sequence_length))

    def predict(self, reader, start, length):
        start = start.lower()
        start_ix = reader.char_to_ix[start]
        x = np.zeros(self._vocab_size)
        x[start_ix] = 1
        ixes = [start_ix]
        hprev = np.zeros(self._hidden_size)
        for t in range(length):
            hprev = np.tanh(self._Wxh@x+self._Whh@hprev+self._bh) 
            o = self._softmax(self._Who@hprev+self._bo)
            ix = np.random.choice(range(self._vocab_size), p=o)
            ixes.append(ix)
            x = np.zeros(self._vocab_size)
            x[ix] = 1
        text = ' '.join(reader.ix_to_char[ix] for ix in ixes)
        text = re.sub(R'([\"\']) ([A-z]+[-\']?[A-z]+) ([\"\'])', r'\1\2\3', text)
        text = re.sub(R'([A-z\'\"]) ([:,.?!])', r'\1\2', text)
        return text.capitalize()

In [178]:
%%script false --no-raise-error
fpath = 'shakespeare.txt'
reader = Reader(fpath)
print(f'{reader.vocab_size} unique words')
rnn = RNN(vocab_size=reader.vocab_size,hidden_size=500)
rnn.fit(reader=reader, epoch=20)